In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from random import choice
import random
import networkx as nx
import os
from pandas.io.json import json_normalize

In [ ]:
# SET CHANNEL PREFERENCES
connectedness_percentile = 50
num_channels_to_make = 3
channel_capacity_sats = 20000

In [ ]:
# GET DATA
lightning_dir = "~/lightning/cli/"
save_dir = "> /Users/dariuscognac/Documents/GitHub/Lightning-Network-Topological-Analysis/"
listnodes = lightning_dir + "lightning-cli listnodes" + save_dir + "list_of_nodes.json"
listchannels = lightning_dir + "lightning-cli listchannels" + save_dir + "list_of_channels.json"

os.system(listnodes);
os.system(listchannels);


# LOAD AND FORMAT DATA
nodes_temp = pd.read_json('list_of_nodes.json')
nodes_table = json_normalize(nodes_temp['nodes'])

channels_temp = pd.read_json('list_of_channels.json')
channels_table = json_normalize(channels_temp['channels'])


# MAKE GRAPH
G = nx.Graph()
G.add_nodes_from(nodes_table['nodeid'])
edges_list = [(channels_table['source'][i], channels_table['destination'][i]) for i in range(len(channels_table))]
G.add_edges_from(edges_list)

print('Number of nodes = ' + str(len(G.nodes())))
print('Number of edges (payment channels) = ' + str(len(G.edges())))


# Get just the main graph
def get_main_subgraph(G):
    all_sub_G = list(nx.connected_component_subgraphs(G))
    largest_sg = 0
    for i, sg in enumerate(all_sub_G):
        if sg.number_of_nodes() > largest_sg:
            largest_sg = sg.number_of_nodes()
            main_G = sg
    return main_G


# Define find_nodes_with_highest_deg_cent()
def find_nodes_high_percentile_deg_cent(G, connectedness_percentile):
    # Compute the degree centrality of G: deg_cent      
    deg_cent = nx.degree_centrality(G)
    # Compute the X percentile of degree centrality
    dc = list(deg_cent.values())
    high_percentile_dc = np.percentile(dc, connectedness_percentile)
    nodes = set()
    # Iterate over the degree centrality dictionary
    for k, v in deg_cent.items():
        # Check if the current value has the maximum degree centrality
        if v > high_percentile_dc:
            # Add the current node to the set of nodes
            nodes.add(k)
    return nodes

# PICK NEW NEIGHBORS
# Remove disconnected nodes
main_G = get_main_subgraph(G)
# Find nodes within top X% of highest centrality
potential_neighbors = list(find_nodes_high_percentile_deg_cent(main_G, connectedness_percentile))
# pick new neighbors
new_neighbors = random.sample(potential_neighbors, num_channels_to_make)


# DISPLAY NEW NEIGHBOR INFO
print("Do you want to connect to these nodes?\n\n")
node_alias=[]; num_channels=[]; ip_address=[]

for i in range(len(new_neighbors)):
    nd = nodes_table[nodes_table['nodeid']==new_neighbors[i]]
    node_alias.append(str(nd['alias']))
    num_channels.append(len(list(main_G.neighbors(new_neighbors[i]))))
    ip_address.append(list(nd['addresses'])[0][0]['address'])
    print("node ID: "+new_neighbors[i])
    print("node alias: "+node_alias[i])
    print("number of channels: "+str(num_channels[i])+"\n")

In [ ]:
for i in range(len(new_neighbors)):

    print("Setting up payment channel with "+node_alias[i])
    connect = lightning_dir+"lightning-cli connect "+new_neighbors[i]+"@"+ip_address[i]
    print(connect)
    # os.system(connect);

    fund_channel = lightning_dir+"lightning-cli fundchannel "+new_neighbors[i]+" "+str(channel_capacity_sats)
    print(fund_channel)
    print("\n")
    # os.system(fund_channel);